In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import scipy.io
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import random
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.nn import init

/Users/mclinwong/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
matpath = '/Users/mclinwong/GitHub/CodesReproduction/DCN-DOA/Data/matlib/'
read_temp=scipy.io.loadmat(matpath + 'data2_trainlow.mat')
S_est=read_temp['S_est']                   #shape:(19800, 120, 2)
#S_abs=read_temp['S_abs']                   #shape:(19800,240)
S_label=read_temp['S_label']               #(19800,120)                     
S_label1 = np.expand_dims(S_label, 2)      #(19800,120,1)

In [3]:
nb_epoch=300
batch_size=120
batch_size_test = 60

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
S_est_train, S_est_test, S_label1_train, S_label1_test = train_test_split(S_est, S_label1, test_size=0.2)

In [6]:
#将训练集和测试集转为Tensor形式，并交换后两维度次序
S_label1_trainTH = torch.Tensor(S_label1_train).permute(0,2,1)
S_label1_testTH = torch.Tensor(S_label1_test).permute(0,2,1)
S_est_trainTH = torch.Tensor(S_est_train).permute(0,2,1)
S_est_testTH = torch.Tensor(S_est_test).permute(0,2,1)

S_est_trainTH = S_est_trainTH.to(device)
S_est_testTH = S_est_testTH.to(device)
S_label1_trainTH = S_label1_trainTH.to(device) 
S_label1_testTH = S_label1_testTH.to(device)

In [7]:
#将数据和标签一一对应
a = ()
for i in range(0,15840):
    a += ((S_est_trainTH[i], S_label1_trainTH[i]),)

b = ()
for i in range(0,3960):
    b += ((S_est_testTH[i], S_label1_testTH[i]),)

In [8]:
train_loader = torch.utils.data.DataLoader(a, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(b, batch_size=batch_size_test, shuffle=False)

In [9]:
#CNN网络结构
class CNN(nn.Module):
    def __init__(self,activ):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv1d(2,12,kernel_size=25, padding=12), nn.BatchNorm1d(12), activ())
        self.conv2 = nn.Sequential(nn.Conv1d(12,6,kernel_size=15, padding=7),  nn.BatchNorm1d(6), activ())
        self.conv3 = nn.Sequential(nn.Conv1d(6,3,kernel_size=5, padding=2), nn.BatchNorm1d(3), activ())
        self.conv4 = nn.Sequential(nn.Conv1d(3,1,kernel_size=3, padding=1),nn.BatchNorm1d(1), activ())
    
    def forward(self, x):  
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
       
        return out

In [10]:
class CNN_resnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv1d(2,12,kernel_size=25, padding=12), nn.BatchNorm1d(12))
        self.conv2 = nn.Sequential(nn.Conv1d(12,6,kernel_size=15, padding=7),  nn.BatchNorm1d(6))
        self.conv3 = nn.Sequential(nn.Conv1d(6,3,kernel_size=5, padding=2), nn.BatchNorm1d(3))
        self.conv4 = nn.Sequential(nn.Conv1d(3,1,kernel_size=3, padding=1),nn.BatchNorm1d(1))
        self.trans1 = nn.Conv1d(2,6,kernel_size=1, padding=0)
        self.trans2 = nn.Conv1d(6,1,kernel_size=1, padding=0)
    
    def forward(self, x):  
        y1 = F.relu(self.conv1(x))
        y2 = F.relu(self.trans1(x) + self.conv2(y1))
        y3 = F.relu(self.conv3(y2))
        y4 = F.relu(self.trans2(y2)+self.conv4(y3))
       
        return y4

In [11]:
def weigth_init(m):
    if isinstance(m, nn.Conv1d):
        init.kaiming_uniform_(m.weight.data)
        init.constant_(m.bias.data,0.1)
    elif isinstance(m, nn.BatchNorm1d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()
#   elif isinstance(m, nn.Linear):
#      m.weight.data.normal_(0,0.01)
#       m.bias.data.zero_()


In [12]:
cnn_relu = CNN(nn.ReLU).to(device)
cnn_tanh = CNN(nn.Tanh).to(device)
cnn_sigmoid = CNN(nn.Sigmoid).to(device)
cnn_relu_resnet = CNN_resnet().to(device)
cnn_tanh_resnet = CNN_resnet().to(device)
cnn_sigmoid_resnet = CNN_resnet().to(device)



cnn_relu.apply(weigth_init)
#cnn_tanh.apply(weigth_init)
#cnn_sigmoid.apply(weigth_init)

CNN(
  (conv1): Sequential(
    (0): Conv1d(2, 12, kernel_size=(25,), stride=(1,), padding=(12,))
    (1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv1d(12, 6, kernel_size=(15,), stride=(1,), padding=(7,))
    (1): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv3): Sequential(
    (0): Conv1d(6, 3, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv4): Sequential(
    (0): Conv1d(3, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
)

In [13]:
loss_func = torch.nn.MSELoss()

In [14]:
#定义训练过程

def train(epoch, model):

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
   
    for  batch_idx, (data, target) in enumerate(train_loader):
        
        data = data.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()        
#        data, target = Variable(data), Variable(target)
        output = model(data)
        
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
    return loss.item()


    print('Train Epoch: {} \tloss: {:.4f}'.format(
                epoch+1, loss.item()
            ))



In [15]:
def test_rmse(model):
    
    a = 0
    
    for (data,label) in test_loader: 
        c = model(data)
        c1 = c.cpu().detach()
        labe = label.cpu()
        
        for i in range(0, 60):
            bias = c1[i,0] - labe[i,0]
            bias_norm = torch.norm(bias, p=1)
            bias_norm_sqr = torch.pow(bias_norm,2)
            a += bias_norm_sqr
            
    
    RMSE = torch.sqrt(a/(3960*2))
    
    return RMSE

In [16]:
#cnn+relu
cnnrele_loss_list = []
for epoch in range(0,300):
    cnn_loss = train(epoch, cnn_relu)
    cnnrele_loss_list.append(cnn_loss)
      
test_rmse(cnn_relu)

KeyboardInterrupt: 

In [ ]:
plt.plot(range(nb_epoch), train_loss, label='train loss')

In [ ]:
#cnn+tanh
for epoch in range(0,300):
    
    train(epoch, cnn_tanh)
    
test_rmse(cnn_tanh)

In [ ]:
#cnn+sigmoid
for epoch in range(0,300):
    
    train(epoch, cnn_sigmoid)
    
test_rmse(cnn_sigmoid)

In [ ]:
#cnn+relu+resnet
for epoch in range(0,300):
    
    train(epoch, cnn_relu_resnet)
    
test_rmse(cnn_relu_resnet)


In [ ]:
read_temp=scipy.io.loadmat(r"C:\Users\Administrator\Desktop\DEEPDOA\code\data2test\data2_test.mat")
K=2
k=1

S_est=read_temp['S_est']
S_label=read_temp['S_label']
R_est=read_temp['R_est']
DOA_train=read_temp['DOA_train']
theta=read_temp['theta']

In [ ]:
S_est_TH = torch.Tensor(S_est).permute(0,2,1)
S_est_TH = S_est_TH.to(device)
S_label_TH = torch.Tensor(S_label)

a = cnn_relu(S_est_TH)
a_1 = a[1,0]
a_2 = np.array(a_1.cpu().detach())
DCN1=np.zeros((120))
DCN1[:]=a_2

b = cnn_tanh(S_est_TH)
b_1 = b[1,0]
b_2 = np.array(b_1.cpu().detach())
DCN2=np.zeros((120))
DCN2[:]=b_2

c = cnn_sigmoid(S_est_TH)
c_1 = c[1,0]
c_2 = np.array(c_1.cpu().detach())
DCN3=np.zeros((120))
DCN3[:]=c_2

d = cnn_relu_resnet(S_est_TH)
d_1 = d[1,0]
d_2 = np.array(d_1.cpu().detach())
DCN4=np.zeros((120))
DCN4[:]=d_2

In [ ]:
plt.rcParams['font.sans-serif']=['Microsoft YaHei']
figsize = 15,3
figure, ax = plt.subplots(figsize=figsize)

plt.subplot(131)
plt.plot(theta.T,(DCN1[:]),linewidth=2.0)
plt.ylim([-0.1,0.8])
plt.plot(DOA_train[:,k],np.ones((K,))*0.6,'rD')
plt.tick_params(labelsize=13)
plt.xlabel('到达波方向($^o$)')
plt.ylabel('频 谱') 
plt.legend(["relu"],loc='upper left')
#plt.show()

plt.subplot(132)
plt.plot(theta.T,(DCN2[:]),linewidth=2.0)
plt.ylim([-0.1,0.8])
plt.plot(DOA_train[:,k],np.ones((K,))*0.6,'rD')
plt.tick_params(labelsize=13)
#labels = ax.get_xticklabels() + ax.get_yticklabels()
plt.xlabel('到达波方向($^o$)')
plt.legend(["tanh"],loc='upper left')
#plt.ylabel('频 谱') 
#plt.show()

plt.subplot(133)
plt.plot(theta.T,(DCN3[:]),linewidth=2.0)
plt.ylim([-0.1,0.8])
plt.plot(DOA_train[:,k],np.ones((K,))*0.6,'rD')
plt.tick_params(labelsize=13)
#labels = ax.get_xticklabels() + ax.get_yticklabels()
plt.xlabel('到达波方向($^o$)')
#plt.ylabel('频 谱') 
plt.legend(["sigmoid"],loc='upper left')
#plt.show()



In [ ]:
figsize = 10,3
figure, ax = plt.subplots(figsize=figsize)

plt.subplot(121)
plt.plot(theta.T,(DCN1[:]),linewidth=2.0)
plt.ylim([-0.1,0.8])
plt.plot(DOA_train[:,k],np.ones((K,))*0.6,'rD')
plt.tick_params(labelsize=13)
plt.xlabel('到达波方向($^o$)')
plt.ylabel('频 谱') 
plt.legend(["CNN+relu"],loc='upper left')

plt.subplot(122)
plt.plot(theta.T,(DCN4[:]),linewidth=2.0)
plt.ylim([-0.1,0.8])
plt.plot(DOA_train[:,k],np.ones((K,))*0.6,'rD')
plt.tick_params(labelsize=13)
#labels = ax.get_xticklabels() + ax.get_yticklabels()
plt.xlabel('到达波方向($^o$)')
plt.legend(["ResNet+relu"],loc='upper left')
#plt.ylabel('频 谱') 
#plt.show()